# Query Data with AWS Data Wrangler

**AWS Data Wrangler** is an open-source Python library that extends the power of the Pandas library to AWS connecting DataFrames and AWS data related services (Amazon Redshift, AWS Glue, Amazon Athena, Amazon EMR, Amazon QuickSight, etc).

* https://github.com/awslabs/aws-data-wrangler
* https://aws-data-wrangler.readthedocs.io

Built on top of other open-source projects like Pandas, Apache Arrow, Boto3, s3fs, SQLAlchemy, Psycopg2 and PyMySQL, it offers abstracted functions to execute usual ETL tasks like load/unload data from Data Lakes, Data Warehouses and Databases.

_Note that AWS Data Wrangler is simply a Python library that uses existing AWS Services.  AWS Data Wrangler is not a separate AWS Service.  You install AWS Data Wrangler through `pip install` as we will see next._

# _Pre-Requisite: Make Sure You Created an Athena Table for Both TSV and Parquet in Previous Notebooks_

In [1]:
%store -r ingest_create_athena_table_tsv_passed

In [2]:
try:
    ingest_create_athena_table_tsv_passed
except NameError:
    print('++++++++++++++++++++++++++++++++++++++++++++++')
    print('[ERROR] YOU HAVE TO RUN ALL PREVIOUS NOTEBOOKS.  You did not register the TSV Data.')
    print('++++++++++++++++++++++++++++++++++++++++++++++')

In [3]:
print(ingest_create_athena_table_tsv_passed)

True


In [4]:
if not ingest_create_athena_table_tsv_passed:
    print('++++++++++++++++++++++++++++++++++++++++++++++')
    print('[ERROR] YOU HAVE TO RUN ALL PREVIOUS NOTEBOOKS.  You did not register the TSV Data.')
    print('++++++++++++++++++++++++++++++++++++++++++++++')
else:
    print('[OK]')

[OK]


In [5]:
%store -r ingest_create_athena_table_parquet_passed

In [6]:
try:
    ingest_create_athena_table_parquet_passed
except NameError:
    print('++++++++++++++++++++++++++++++++++++++++++++++')
    print('[ERROR] YOU HAVE TO RUN ALL PREVIOUS NOTEBOOKS.  You did not convert into Parquet data.')
    print('++++++++++++++++++++++++++++++++++++++++++++++')

In [7]:
print(ingest_create_athena_table_parquet_passed)

True


In [8]:
if not ingest_create_athena_table_parquet_passed:
    print('++++++++++++++++++++++++++++++++++++++++++++++')
    print('[ERROR] YOU HAVE TO RUN ALL PREVIOUS NOTEBOOKS.  You did not convert into Parquet data.') 
    print('++++++++++++++++++++++++++++++++++++++++++++++')
else:
    print('[OK]')

[OK]


# Setup

In [9]:
import sagemaker
import boto3

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = sagemaker_session.default_bucket()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name='sagemaker', region_name=region)

In [10]:
import awswrangler as wr

# Query Parquet from S3 with Push-Down Filters

Read Apache Parquet file(s) from from a received S3 prefix or list of S3 objects paths.

The concept of Dataset goes beyond the simple idea of files and enable more complex features like partitioning and catalog integration (AWS Glue Catalog): 

_dataset (bool)_ – If True read a parquet dataset instead of simple file(s) loading all the related partitions as columns.

In [11]:
p_filter = lambda x: True if x["product_category"] == "Digital_Software" else False

In [12]:
path = 's3://{}/amazon-reviews-pds/parquet/'.format(bucket)
df_parquet_results = wr.s3.read_parquet(path,
                                        columns=['star_rating', 'product_category', 'review_body'],
                                        partition_filter=p_filter,
                                        dataset=True)
df_parquet_results.shape

(102084, 3)

In [13]:
df_parquet_results.head(5)

,star_rating,review_body,product_category
0,2,"Did not have all the forms, I was missing city...",Digital_Software
1,5,Norton Antivirus PC download is reasonably pri...,Digital_Software
2,1,Do not waste your time trying to purchase to d...,Digital_Software
3,5,This program has worked great on my Mac with n...,Digital_Software
4,2,It really isn't a true sync of my Quicken data...,Digital_Software


# Query Parquet from S3 in Chunks

Batching (chunked argument) (Memory Friendly):

Will enable the function to return a Iterable of DataFrames instead of a regular DataFrame.

There are two batching strategies on Wrangler:
* If chunked=True, a new DataFrame will be returned for each file in your path/dataset.
* If chunked=INTEGER, Wrangler will iterate on the data by number of rows equal to the received INTEGER.

P.S. chunked=True if faster and uses less memory while chunked=INTEGER is more precise in number of rows for each Dataframe.

In [14]:
path = 's3://{}/amazon-reviews-pds/parquet/'.format(bucket)
chunk_iter = wr.s3.read_parquet(path,
                                columns=['star_rating', 'product_category', 'review_body'],
                                # filters=[("product_category", "=", "Digital_Software")],
                                partition_filter=p_filter,
                                dataset=True,
                                chunked=True)

In [15]:
print(next(chunk_iter))

     star_rating                                        review_body  \
0              2  Did not have all the forms, I was missing city...   
1              5  Norton Antivirus PC download is reasonably pri...   
2              1  Do not waste your time trying to purchase to d...   
3              5  This program has worked great on my Mac with n...   
4              2  It really isn't a true sync of my Quicken data...   
..           ...                                                ...   
186            4  This is a well thought out program and has eve...   
187            5  Suits my needs very well. Thanks but I wish I ...   
188            4  When it came time to renew my internet protect...   
189            5  We have investments in several MLP's that send...   
190            5  I am a sole proprietor with a very small busin...   

     product_category  
0    Digital_Software  
1    Digital_Software  
2    Digital_Software  
3    Digital_Software  
4    Digital_Software  
.. 

# Query the Glue Catalog (ie. Hive Metastore)
Get an iterator of tables.

In [16]:
database_name = 'dsoaws'
table_name_tsv = 'amazon_reviews_tsv'
table_name_parquet = 'amazon_reviews_parquet'

In [17]:
for table in wr.catalog.get_tables(database="dsoaws"):
    print(table['Name'])

amazon_reviews_parquet
amazon_reviews_tsv


# Query from Athena
Execute any SQL query on AWS Athena and return the results as a Pandas DataFrame.  


In [18]:
%%time
df = wr.athena.read_sql_query(
    sql='SELECT * FROM {} LIMIT 5000'.format(table_name_parquet),
    database=database_name
)

CPU times: user 259 ms, sys: 21.7 ms, total: 281 ms
Wall time: 6.71 s


In [19]:
df.head(5)

,marketplace,customer_id,review_id,product_id,product_parent,product_title,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,year,review_date,product_category
0,US,28931209,RBNC2XLX461DQ,B009HBCZPQ,808553197,Turbo Tax Parent V2,2,0,1,N,Y,Not worth the money,"Did not have all the forms, I was missing city...",2013,2013-04-20,Digital_Software
1,US,52887452,RKH9IGDFBFVHD,B008SCNCTI,866682919,Norton Antivirus 2013 - 1 User / 3 PC,5,0,0,N,Y,All around good product,Norton Antivirus PC download is reasonably pri...,2013,2013-04-20,Digital_Software
2,US,17878349,R1AY7RGUK9LJA7,B009VI8E82,302765791,Microsoft Windows 8 Pro Pack (Win 8 to Win 8 P...,1,1,4,N,Y,Smoke and Mirrors,Do not waste your time trying to purchase to d...,2013,2013-04-20,Digital_Software
3,US,28404410,R2XB4JF74JWIC2,B0064PFB9U,232554866,Office Mac Home and Student 2011 - 1PC/1User [...,5,0,0,N,Y,Works Great on My Mac,This program has worked great on my Mac with n...,2013,2013-04-20,Digital_Software
4,US,16925331,R1SPSWOA91HMPK,B008S0IMCC,534964191,Quicken Deluxe 2013,2,0,0,N,Y,Needs work,It really isn't a true sync of my Quicken data...,2013,2013-04-20,Digital_Software


# Query from Athena in Chunks
Retrieving in chunks can help reduce memory requirements.  

_This will take a few seconds._

In [20]:
%%time

chunk_iter = wr.athena.read_sql_query(
    sql='SELECT * FROM {} LIMIT 5000'.format(table_name_parquet),
    database='{}'.format(database_name),
    chunksize=64_000  # 64 KB Chunks
)

CPU times: user 203 ms, sys: 12 ms, total: 216 ms
Wall time: 6.74 s


In [21]:
print(next(chunk_iter))

     marketplace customer_id       review_id  product_id product_parent  \
0             US    51106503  R3RO07ID7KIX9Y  B00GUXLRCQ      387279151   
1             US    17190816  R2G4LXBHM6DQ65  B00H9A60O4      608720080   
2             US    42984321   RAMRN0WMX1CYR  B009KTSKVI      369346636   
3             US    22028396  R23B8TRY9BY1C2  B00F8LJXKY       53079259   
4             US    39853265  R135YS3STI0G2A  B00FFINFM2      218020461   
...          ...         ...             ...         ...            ...   
4679          US    45483279  R2VY5ZTR0TRKLR  B002ASAGSG      884182111   
4680          US    37622817  R1RRSSCMGAX6K9  B009SPZ11Q      826849966   
4681          US    14892143  R21W37UR310AC3  B00GUNLZ00      312588924   
4682          US    13236238  R1G19RUCIVIC9X  B00BLVM00A      880777294   
4683          US    12839079  R1V5732BMMPKZX  B00GGUUFUI      721570092   

                                          product_title  star_rating  \
0                          

In [ ]:
%%javascript
Jupyter.notebook.save_checkpoint();
Jupyter.notebook.session.delete();